# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2025/11/11/

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company/brand page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Qwen/Qwen-Image-2512
Updated
6 days ago
•
14.3k
•
470
tencent/HY-MT1.5-1.8B
Updated
5 days ago
•
5.59k
•
410
LGAI-EXAONE/K-EXAONE-236B-A23B
Updated
about 11 hours ago
•
2.06k
•
389
IQuestLab/IQuest-Coder-V1-40B-Loop-Instruct
Updated
3 days ago
•
5.2k
•
247
MiniMaxAI/MiniMax-M2.1
Updated
10 days ago
•
197k
•
889
Browse 2M+ models
Spaces
Running
Featured
3.63k
Wan2.2 Animate
👁
3.63k
Wan2.2 Animate
Running
on
Zero
MCP
Featured
226
Qwen-Image-Edit-2511-LoRAs-Fast
🎃
226
Demo of the Collection of Qwen Image Edit LoRAs
Running
on
Zero
1.05k


In [23]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen-Image-2512\nUpdated\n6 days ago\n•\n14.3k\n•\n471\ntencent/HY-MT1.5-1.8B\nUpdated\n5 days ago\n•\n5.59k\n•\n425\nLGAI-EXAONE/K-EXAONE-236B-A23B\nUpdated\nabout 11 hours ago\n•\n2.06k\n•\n389\nIQuestLab/IQuest-Coder-V1-40B-Loop-Instruct\nUpdated\n3 days ago\n•\n5.2k\n•\n247\nMiniMaxAI/MiniMax-M2.1\nUpdated\n10 days ago\n•\n197k\n•\n889\nBrowse 2M+ models\nSpaces\nRunning\nFeatured\

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a leading AI community and collaboration platform dedicated to building the future of machine learning (ML). It serves as a central hub where the global machine learning community collaborates on open-source models, datasets, and applications. With over 2 million models and 500,000 datasets, Hugging Face empowers researchers, developers, and companies to create and share AI technologies with ease.

---

## What We Offer

- **Models:** Access to a vast repository of 2 million+ machine learning models spanning text, image, video, audio, and even 3D data.
- **Datasets:** Explore more than 500,000 datasets suitable for diverse ML tasks and research.
- **Spaces:** Interactive ML applications and demos hosted by the community, fostering experimentation and innovation.
- **Community:** A fast-growing and vibrant community where ML practitioners, scientists, and engineers learn, collaborate, and contribute.
- **Enterprise Solutions:** Paid Compute and customized team & enterprise solutions for advanced AI development and deployment.
- **Open Source Stack:** Accelerate your projects by leveraging the Hugging Face open source tools and infrastructure.

---

## Company Culture

Hugging Face is committed to:

- **Open and Ethical AI:** Building a future with AI that is transparent, ethical, and accessible.
- **Collaboration:** Fostering global teamwork by providing a platform where anyone can share and discover ML advancements.
- **Innovation:** Supporting cutting-edge research and applications through a dynamic ecosystem.
- **Empowerment:** Enabling individuals and teams to grow their skills, build portfolios, and contribute to AI progress.

---

## Who Uses Hugging Face?

- **Researchers and Data Scientists:** Access and share state-of-the-art models and datasets for academic and applied research.
- **Machine Learning Engineers:** Build and deploy AI models faster with an extensive community and tools.
- **Enterprises:** Launch AI-driven products and solutions using scalable and secure enterprise services.
- **AI Developers and Hobbyists:** Experiment and showcase their work on the platform while receiving community feedback.

---

## Careers & Opportunities

Hugging Face is continuously expanding its team and welcomes passionate individuals interested in making an impact on the future of AI. The company values:

- Expertise in machine learning, software development, and AI research.
- A collaborative mindset and a drive to build open, ethical AI.
- Enthusiasm for working in a fast-paced, innovative, and community-driven environment.

Explore career openings by visiting the Hugging Face website and join a team that’s shaping the future of AI technology.

---

## Join Us Today

Whether you are developing the next breakthrough model, searching for datasets, or looking to collaborate with like-minded AI enthusiasts, Hugging Face provides the tools, community, and enterprise solutions you need to succeed.

**Visit:** [huggingface.co](https://huggingface.co)  
**Sign Up for Free** to start exploring over 2 million models and 500,000 datasets now.

---

### Brand Colors & Identity

- Signature Yellow: #FFD21E  
- Orange Accent: #FF9D00  
- Neutral Gray: #6B7280

Hugging Face stands out with a bright and approachable brand that reflects its commitment to openness and innovation in AI.

---

**Hugging Face — The AI community building the future.**

In [21]:
create_brochure("The Car Expert", "https://www.thecarexpert.co.uk/")

Selecting relevant links for https://www.thecarexpert.co.uk/ by calling gpt-5-nano
Found 12 relevant links


# The Car Expert

---

## About Us

The Car Expert is the UK’s leading independent source of car buying and finance advice. With a passionate team of automotive experts, we are committed to empowering consumers with trustworthy, clear, and unbiased information to make the best car decisions. Whether you're buying your first car, upgrading to the latest electric model, leasing, or insuring your vehicle, The Car Expert provides comprehensive guidance every step of the way.

---

## What We Offer

### Expert Advice

- **Buying and Selling**: Detailed guides and tips on car buying and selling, including how to find your perfect car and the best ways to sell your vehicle.
- **Finance & Leasing**: Clear explanations of all car finance options, including PCP, HP, PCH, salary sacrifice schemes, and subscriptions.
- **Ownership**: Practical advice on car insurance, GAP insurance, warranties, maintenance, care, safety, and technology.
- **Driving**: Resources covering driving tips, car tyre maintenance, and emerging vehicle technologies.

### Expert Ratings & Reviews

- Comprehensive expert ratings across all car segments: from small city cars to luxury SUVs and high-performance vehicles.
- Annual “Best Cars” lists such as the best small cars, best SUVs, best electric vehicles, and more.
- In-depth reviews of new cars and automotive products to help consumers make informed choices.

### Community & News

- Active **Forum** where car enthusiasts and buyers share experiences and advice.
- Up-to-date **Car Manufacturer News** and industry insights.
- Access to newsletters and media content including focused verticals like The Van Expert and The Truck Expert.

---

## Our Customers

We serve a diverse range of customers including:

- First-time car buyers needing guidance on purchasing and financing.
- Experienced car owners looking for trusted maintenance and insurance advice.
- Car enthusiasts seeking expert opinions on the latest models and technologies.
- Anyone seeking unbiased, easy-to-understand car finance and leasing options.

Our mission is to make car ownership transparent, simple, and financially sensible for consumers across the UK.

---

## Company Culture

- **Customer-Centric**: We believe in putting our users first by delivering expert advice that is clear, honest, and actionable.
- **Transparency & Trust**: Independence from car dealers and manufacturers ensures our content is unbiased.
- **Passion for Cars**: Our team shares a genuine passion for all things automotive, driving us to continually update and expand our expertise.
- **Continuous Learning**: We embrace new technologies and innovations in the car industry to keep our community well informed.
- **Collaboration & Community**: We foster an open, supportive environment encouraging community engagement and expert interaction through forums and social media.

---

## Careers at The Car Expert

At The Car Expert, we are always looking for talented individuals who share our passion for cars and helping consumers. Career opportunities may include roles in:

- Automotive journalism and content creation
- Car finance and leasing expertise
- Digital marketing and SEO
- Community management
- Technical development and data analysis

Join us to be part of an innovative, growth-oriented company dedicated to excellence in automotive advice and consumer service.

---

## Connect With Us

Stay updated and join the conversation:

- **Facebook:** [The Car Expert](https://facebook.com/thecarexpert)
- **Instagram:** [@thecarexpert](https://instagram.com/thecarexpert)
- **LinkedIn:** [The Car Expert](https://linkedin.com/company/thecarexpert)
- **Twitter:** [@thecarexpert](https://twitter.com/thecarexpert)
- **YouTube:** [The Car Expert Channel](https://youtube.com/thecarexpert)

Visit us at [thecarexpert.co.uk](https://thecarexpert.co.uk) to explore expert advice, reviews, ratings, and forums.

---

**The Car Expert — Your trusted guide for all things car finance, buying, and ownership.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("Thinkanew Media Ltd", "https://www.thinkanewmedia.com/")

Selecting relevant links for https://www.thinkanewmedia.com/ by calling gpt-5-nano
Found 8 relevant links


# Welcome to Thinkanew Media Ltd – Where Your Digital Dreams Get Wired to Reality!

## Who Are We?
Based in the heart of Surrey, Thinkanew Media Ltd is your friendly neighborhood web design and application development agency that takes your business ideas and catapults them into the digital stratosphere. Whether you’re dreaming of a sleek WordPress website or a lightning-fast GatsbyJS-powered masterpiece, we're the tech aficionados ready to make it happen.

## What Do We Do? (Spoiler: We’re Awesome at Web Stuff)
- **Web Design & Development:** Custom-built WordPress sites for flexibility and value, or GatsbyJS sites for when you want your website to run like a caffeinated cheetah — blazing fast **and** SEO-friendly.
- **Web Application Development:** Full-stack web apps for businesses of all sizes. Need backend wizardry or front-end magic? We’ve got the spells.
- **Support & Maintenance:** Your digital babies need care after launch. We keep your websites and apps secure, fast, and happy.
- **Hosting & Email:** Rugged, reliable, and secure hosting tailored to your needs. Plus, we set you up with pro email signatures (because your emails deserve to look as sharp as you do).
- **IT Consulting:** Confused about hardware, software, or online meetings? We’ll guide you through the tech jungle with a map and compass in hand.

## Our Culture – Geek, Chic & Cheerful
At Thinkanew Media, we blend technical expertise with a collaborative “let’s build something awesome together” spirit. We believe the best tech solutions are born out of fun, fresh ideas, and a dash of curiosity. Our team is passionate about WordPress — 10+ years of love — and we champion modern frameworks like GatsbyJS to keep you ahead of the pack.

We’re not just a bunch of code monkeys; we’re your strategic partners, problem solvers, and occasional meme sharers. If you enjoy innovation, quality, and a good laugh, you’ll fit right in.

## Who We Serve
From scrappy startups to savvy established businesses across Surrey (and beyond), we pride ourselves on tailored solutions that power growth and digital connections. Our clients want performance, reliability, and creativity — and we deliver all three, often with a side of tech wizardry.

## Careers at Thinkanew Media – Join Our Digital Dream Team!
Fancy a career where your coding skills don’t just sit in a cubicle closet but shine out loud? Whether you’re a WordPress wizard, a GatsbyJS guru, or a full-stack superstar craving real impact, Thinkanew Media is the place to be. We value curiosity, collaboration, and pushing digital boundaries.

### What You Can Expect:
- Diverse projects with real clients hungry for digital excellence.
- A culture that values your ideas (even the outlandish ones).
- Opportunities to grow, learn, and geek out with fellow tech lovers.
- Flexible work environment because life happens outside the keyboard too.

## Ready to Make Your Mark in the Digital World?
Thinkanew Media is here to help you blaze a trail, break the internet (in a good way), and create digital connections that count. Whether you want us to craft your website, build robust apps, keep your tech humming, or simply have a chat about your digital needs — reach out. We promise, no boring tech jargon unless you really want it.

---

**Thinkanew Media Ltd**  
Expert Web Design & Development Agency — Surrey's Best-Kept Digital Secret  
[Get in Touch] | [Arrange a Call] 

*Registered in England & Wales. Company No. 7142861. Privacy policy applies.* 

---

### Thinkanew Media — We Bring Your Ideas to Life… One Pixel, Byte, and Laugh at a Time!

In [25]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Welcome to Hugging Face — Where AI Gets Its Hug On 🤗

---

## Who We Are

Hugging Face is not just a company; it’s the **bustling AI community** buzzing with more than 2 million machine learning models, over half a million datasets, and a vibrant crowd of developers, researchers, and AI enthusiasts all cozying up to build the future of artificial intelligence—together. Think of us as the ultimate group chat for ML geeks, only instead of memes, we share state-of-the-art models and groundbreaking datasets.

> “Building the future” isn’t just a tagline. It’s our daily workout.

---

## What We Offer

### The Hugging Face Hub

- **Models?** We got models. 2 MILLION+ models from image generators to language wizards are all waiting for you to explore or contribute. Want photorealistic images from text? We have that. How about coding assistants? Yup.
- **Datasets?** Over 500,000 datasets spanning text, images, audio, video, and even 3D data. It’s like a candy store for data nerds.
- **Spaces?** Host your apps and demos in our interactive “Spaces.” Whether you want to animate, edit images, or turbocharge your AI innovations, our Spaces are your playground.
- **Community:** A thriving open-source party where everyone shares, experiments, and pushes AI boundaries ethically and collaboratively. No gatekeepers, no secret handshakes—just pure AI love.

---

## Why Hugging Face? Because Everyone Deserves a Hug 🤗

- **Collaboration Central:** Host unlimited public models, datasets, and applications with zero fuss.
- **Cutting-Edge Open Source:** Get your hands on the latest open-source ML stack and move faster than a caffeinated coder.
- **All Modalities Welcome:** Text, image, audio, video, 3D—you name it, we embrace it.
- **Portfolio Building:** Show off your AI skills to the world. Impress your future boss or your cat.
- **Enterprise & Compute Services:** For those who want to scale responsibly with top-notch paid options.

---

## Our Community Culture — Serious AI, Seriously Fun

At Hugging Face, we believe AI development should feel a little less like a cold lab experiment and a bit more like a group project with your best friends (except we don’t make you do all the work). Our culture values:

- **Openness:** Share your models and discoveries freely with the world.
- **Inclusivity:** From newbies to AI ninjas, everyone’s welcome.
- **Innovation with Ethics:** We’re building AI that’s open, fair, and friendly.
- **Playfulness:** Because what’s AI without a little fun?

---

## For Prospective Customers & Enterprise Teams

Ready to supercharge your ML projects? Hugging Face’s **Enterprise Solutions** offer the most advanced, scalable AI platform for your team. Need power? Need speed? Need support? We’ve got you covered.

---

## Careers — Join the AI Hug Squad!

If you have a passion for machine learning, open-source, and community-driven innovation, Hugging Face wants YOU! We’re on the lookout for engineers, researchers, product wizards, and dreamers who:

- Love collaboration over competition.
- Want to help shape the AI landscape ethically.
- Enjoy working in a fast-paced, supportive environment.
- Prefer emojis 🤗 over corporate jargon any day.

Expect excitement, challenges, and if you peek in—maybe even some emoji-filled Slack channels.

---

## Ready to Dive In?

- Explore the future of AI at [huggingface.co](https://huggingface.co)
- Browse 2,000,000+ models, 500,000+ datasets, and a million+ AI apps.
- Join the community that’s literally hugging AI into existence.

---

## Fun Fact

Our logo? A smiling face literally hugging itself. Because at Hugging Face, AI never feels lonely.

---

*Hugging Face — where machine learning models and people come together, and everyone leaves a little smarter (and a lot happier).*

---

**P.S.** We also throw in free virtual hugs. Just ask. 🤗

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>